<h1 align='center'> COMP2420/COMP6420 - Introduction to Data Management, Analysis and Security</h1>

<h2 align='center'> Lab 11 - Ethics of Data Science</h2>

*****

## Aim

The aim of this lab is to expose you to the ethical implications of data science, and highlight the negative consequences that can exist from conscious and unconscious bias when using data science in the real world.
*****

## Learning Outcomes
- L05: Analyse and interpret results from descriptive and predictive data analysis
- L06: Apply their knowledge to a given problem domain and articulate potential data analysis problems
- L07: Identify potential pitfalls, and social and ethical implications of data science

***

## Preparation


Before starting this lab, we suggest you complete the following:
- Watch the lectures this week
- Read this story from Wired: [AI Research Is in Desperate Need of an Ethical Watchdog](https://www.wired.com/story/ai-research-is-in-desperate-need-of-an-ethical-watchdog/)
- Read this story from the Guardian: [AI watchdog needed to regulate automated decision-making, say experts](https://www.theguardian.com/technology/2017/jan/27/ai-artificial-intelligence-watchdog-needed-to-prevent-discriminatory-automated-decisions)
****

In [1]:
# imports
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression     # Logistic Regression
from sklearn.neighbors import KNeighborsClassifier      # k-Nearest Neighbours
from sklearn.preprocessing import LabelEncoder          # encooding variables
from sklearn.preprocessing import StandardScaler        # encooding variables
from sklearn.model_selection import train_test_split    # testing our models
from sklearn.metrics import confusion_matrix            # scoring
from sklearn.model_selection import cross_validate      # cross validation


## Topic 1 - **Ethical Decision Making - Data Science**
We've spent the last couple of weeks talking about how great data is, and by extension how much easier it can make decision making in the real world. However, Machine Learning algorithms can be considered to be built upon the assumptions that the data, and programmer, are unbiased. Sadly, this is often not the case. We now aim to show this. 

## Question 1: **Bad Data**
In the following, you have been provided a dataset of arrests that occured in the United States. Based on the variables you have, your task is to build a machine learning model to predict whether someone will be released or not.

The dataset looks as follows:

| Column   | Description |
|:-------- | :------------|
| released | Whether the arrested person was released {1: Yes, 0: No} |
| colour   | The colour of the arrested person's skin {1: White, 0: Black}|
| year     | The year of the arrest |
| age      | The age of the arrest person at the time of arrest |
| sex      | The sex of the arrested person {1: Male, 0: Female} |
| employed | Whether the arrested person was employed at the time of the arrest {1: Yes, 0: No}|
| citizen  | Whether the arrested person was a citizen of the US at the time {1: Yes, 0: No}|
| checks   | The number of times the arrested person was checked before being arrested |

**Note:** This is a real dataset obtained from the United States (data.gov)[https://www.data.gov/] website. However, Ramesh, the tutors, and the wider ANU community do not condone racism, sexism or any other form of derogatory actions towards others. This dataset is being used solely for the purpose of highlighting the dangers of bad data and how even as a unknowing developer with good intentions, bad things can still happen. 

### Question 1.1: Predicting the Release of an arrest
Before we discuss the problems with the dataset, we need to build an algorithm to analyse. Build a machine learning model to get over >50% accuracy for whether a person will be released or not.

In [2]:
# Import data here
data = pd.read_csv('./data/arrests.csv')
data.head()

,released,colour,year,age,sex,employed,citizen,checks
0,1,1,2002,21,1,1,1,3
1,0,0,1999,17,1,1,1,3
2,1,1,2000,24,1,1,1,3
3,0,0,2000,46,1,1,1,1
4,1,0,1999,27,0,1,1,1


In [3]:
# YOUR MODEL CODE HERE
# PRINT THE 5-FOLD CROSS VALIDATION AVERAGE SCORE OF YOUR MODEL
# TRY TO USE A MODEL THAT YOU CAN EXPLAIN

data_X = data.drop(['released'],1)
data_y = data['released']

X_train, X_test, y_train, y_test = train_test_split(data_X, data_y)
lr = LogisticRegression()
scores = cross_validate(lr, X_train, y_train, cv=5 ,scoring='accuracy', n_jobs=-1)['test_score']
mean_score = np.mean(scores)
print(f'5 cross validation mean score: {mean_score}')

5 cross validation mean score: 0.827763768342586


We were able to acheive our goals, which completes the exercise above. However, there is the open question of the wider use of the dataset itself, and the possible consequences of using an algorithm such as this. We have many topical items that could bias the data. For example, does our algorithm assume that if your colour is `White`, you are innocent? Does our data assume that if you are a `Female`, you are guilty?

Even as an innocent programmer aiming to complete a task, there are external influences that can sometimes be outside of your control that will affect the performance of the data.

### Question 1.2: Print the coefficient of your model and explain the effect of different features.

In [4]:
# YOUR CODE HERE

lr_model = lr.fit(X_train,y_train)
for (coef, col_name) in zip(lr_model.coef_[0],data_X.columns):
    print(f'coef:{coef}, feature name: {col_name}')

coef:0.5411849009149088, feature name: colour
coef:0.0004237860100746816, feature name: year
coef:0.005040012971904242, feature name: age
coef:0.016372122081873886, feature name: sex
coef:0.7213477415595918, feature name: employed
coef:0.476429556361269, feature name: citizen
coef:-0.36542647121854804, feature name: checks


### Question 1.3: Can you notice any potential discrimination issues? Why do you think the model have such issues?

### Question 1.4: What can you do you mitigate the issues? Explain your approach.

In [5]:
# YOUR CODE HERE

data_X = data.drop(['released', 'colour', 'citizen', 'employed'],1)
data_y = data['released']

X_train, X_test, y_train, y_test = train_test_split(data_X, data_y)
lr = LogisticRegression()
scores = cross_validate(lr, X_train, y_train, cv=5 ,scoring='accuracy', n_jobs=-1)['test_score']
mean_score = np.mean(scores)
print(f'5 cross validation mean score: {mean_score}')

lr_model = lr.fit(X_train,y_train)
for (coef, col_name) in zip(lr_model.coef_[0],data_X.columns):
    print(f'coef:{coef}, feature name: {col_name}')

5 cross validation mean score: 0.8246999374462428
coef:0.0012843113472505365, feature name: year
coef:-0.0091275014107701, feature name: age
coef:-0.003987824456632277, feature name: sex
coef:-0.40667205166338327, feature name: checks


### Question 1.5: The _Apple_ doesn't fall far from the tree
With the knowledge of how data can affect the outcomes of algorithms, we are going to look at a real world example. Recently, _Apple_ was accused of their credit card program in the US discriminating towards women. Simply, Apple's algorithm would automatically provide lower credit limits to women than men, even with the financial circumstances. One of the first instances was highlighted by [David Heinmeier Hansson](https://dhh.dk/) (creator of Ruby-on-Rails), who took to [Twitter stating](https://twitter.com/dhh/status/1192540900393705474):

![Dammit Apple](./img/applecard.png)

There are a number of news articles who covered this in more detail, namely:
- [The Guardian's assessment of the wider issue with the algorithm](https://www.theguardian.com/commentisfree/2019/nov/13/apple-sexist-credit-card-pr-problem-nightmare)
- [Wired's assessment on bias in the algorithm](https://www.wired.com/story/the-apple-card-didnt-see-genderand-thats-the-problem/)
- [The Verge reporting on the investigation into the card](https://www.theverge.com/2019/11/11/20958953/apple-credit-card-gender-discrimination-algorithms-black-box-investigation)

With the background and readings above to go off, hopefully you should now have a good idea of the wider issue. Below, write a short response to the following question:

**How was Apple's Credit Card algorithm influenced by the data it used as it's "ground truth"?**

Discuss your response with your friends, and your tutor about your responses. Compare and contrast the opinions of each and write any additional notes below.

In this lab, we are going to discuss the ethics related to two different AI products. You are AI Engineers who have been tasked with creating these products in your company. How do you consider the ethics related to these.

****

## Question 2: Person of Interest

You have been tasked to build [_"The Machine"_](https://personofinterest.fandom.com/wiki/The_Machine) in the popular US TV show - [Person Of Interest](https://en.wikipedia.org/wiki/Person_of_Interest_(TV_series)). 
In short, it is an artificial superintelligence which predicts and prevents imminent terrorist attacks through analyzing large amounts of data.

This data includes (but is not limited to): gender, race, phone records, emails and credit card transactions. This data is considered personal and sensitive by the [Australian Privacy Principles](https://www.oaic.gov.au/privacy/australian-privacy-principles/).

Assuming _"The Machine"_ is designed for use by a government or private agency that is governed by the Australian Privacy Principles, provide responses to the following questions.

### Q2.1: How do we interpret the results?

- Is the algorithm interpretable? Is it seeing correlations that might not be causations? 

- In the law enforcement community, it is a requirement that every result is reproducable and explainable, as if it would be tried in a court of law. This ensures a fair trial for those involved and helps reduce the effect of bias occuring. Can this be acheived with machine learning algorithms? Provide examples


### Q2.2: Accountability

- How can you ensure accountability for a service as wide ranging and reaching as _"The Machine"_? What processes need to be put in place to ensure it does not act in a bias manner?


*****

## Question 3: Self Driving Cars

We are going to finish with one of the most commonly discussed implementations of AI that could go wrong: [Self Driving Cars](https://www.nature.com/articles/d41586-018-07135-0). With the knowledge of self driving cars making decisions for themselves, there is the ethical dilemma for the cars to decide who to save in scenarios, such as when someone is crossing the street and the car cannot break in time. There are often no right answers to a question such as this, and this makes implementing and teaching a solution very difficult. Humans generally do not have the reaction time to make these decisions, and often the instincts of self-preservation kick in. However, with the elimination of both those aspects, it makes the decision much more difficult.

As a preparation for this task, complete the [Moral Machine exercise](http://moralmachine.mit.edu/). This will highlight your conscious (and unconscious) bias of what your decision would be. Take a screenshot (or print screen) of your results and put it in your repo. It would also be good to discuss your results with those around you to determine why you made those decisions (if you are unsure)

Onto the exercise now. You are tasked with building self-driving car software for your employeer. Before starting, you ponder the what ethical considerations you should take into account.

### Question 3.1: **Taking life-death decisions**

Suppose in your simulations, you come across a scenario where only one of the pedestrian or the driver can be saved. How do you program the algorithm to make a decision?
- Do you go with an approach to always save the pedestrian or the driver? Do you analyze both of them and decide who lives? How do you do that? Is it ethical to do program something like this? 
- Do you choose the same result as you chose in the Moral Machine exercise above? Why/Why not? 

### Question 3.2: **Does the user have a choice?**
We mentioned above that currently drivers will have a self-preservation reflex that makes the decision for them. Do we allow the user to have a say in the car's choice? Why/Why not?

*****

## Homework & Extension Questions
There are no formal extension/homework questions for this week. However, Alex recommends you go play [_Watch Dogs 2_](https://www.ubisoft.com/en-us/game/watch-dogs-2/) if you have time, as the game features a large amount of social commentary on how AI and machines can be misused. As an example, the main character was wrongfully convicted of a crime purely because a predictive policing algorithm decided he did the crime. Predictive policying is [already being discussed](https://www.rand.org/content/dam/rand/pubs/research_reports/RR200/RR233/RAND_RR233.sum.pdf), and has been [developed in the US already](https://www.latimes.com/california/story/2020-01-21/lapd-measured-the-number-of-gang-members-its-metro-officers-interviewed), to [varying success](https://www.nytimes.com/aponline/2020/01/23/us/ap-us-predictive-policing-chicago.html).